https://www.wired.com/story/best-wordle-tips/

In [ ]:
%load_ext kedro.ipython

In [ ]:
%reload_kedro

In [3]:
import sys
sys.path.append("../../src/projectwordle")

In [4]:
import polars as pl
import numpy as np
from IPython.display import HTML
from projectwordle.utils import (
    color_pattern_matching,
    difficulty_distribution,
)

pl.Config(tbl_rows=50)

# Load Data

In [ ]:
five_letter_word_anagrams = catalog.load("five_letter_words_anagrams")
alternative_openers = catalog.load("alternative_openers")

In [37]:
alternative_openers.head(6)

index,challenge,guess,match_pattern,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num
u32,str,str,str,str,str,u8,u8,u8,u16,str,bool,str,u32,u8,str,str,u8
0,"""yummy""","""soare""","""BBBBB""","""sreao""","""""",5,0,0,538,"""fully, thick, light, limit, th…",false,"""linty""",0,null,"""very hard""","""soare""",1
1,"""yummy""","""linty""","""BBBBG""","""sitreaoln""","""y""",9,1,1,50,"""fuzzy, puffy, guppy, dummy, mu…",false,"""fuzzy""",0,null,"""very hard""","""linty""",1
2,"""yummy""","""fuzzy""","""BGBBG""","""sitreaofzln""","""yu""",11,2,2,37,"""duchy, buddy, jumpy, mummy, yu…",true,"""duchy""",0,null,"""very hard""","""fuzzy""",1
3,"""yummy""","""duchy""","""BGBBG""","""sitcreaofzdlhn""","""yu""",14,2,2,16,"""puppy, mummy, buggy, bumpy, gu…",true,"""puppy""",0,null,"""very hard""","""duchy""",1
4,"""yummy""","""puppy""","""BGBBG""","""sitcreaopfzdlhn""","""yu""",15,2,2,9,"""mummy, buggy, gummy, muggy, yu…",true,"""mummy""",0,null,"""very hard""","""puppy""",1
5,"""yummy""","""mummy""","""YGGGG""","""sitcreaopfzdlhn""","""yum""",15,3,3,2,"""gummy, yummy""",true,"""gummy""",0,null,"""very hard""","""mummy""",1


In [7]:
alternative_openers.height

54528

In [8]:
simulating_tries_difficulty = (
    alternative_openers
    .select(
        "challenge", "guess", "letter_differences", "common_letters",
        "num_diff_letters", "num_common_letters", "num_matching_index",
        "match_pattern", "num_choices_after_guess", "possible_guesses",
        "challenge_in_possible_guesses", "next_guess", "group", "tries",
        "difficulty", "guess_word_anagrams", "anagram_num"
    )
    .join(
        five_letter_word_anagrams.select("words", "anagrams", "anagram_num"),
        left_on="next_guess",
        right_on="words",
        how="left",
        coalesce=True
    )
)

In [9]:
simulating_tries_difficulty.head()

challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num,anagrams,anagram_num_right
str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u32,u8,str,str,u8,str,u8
"""yummy""","""soare""","""sreao""","""""",5,0,0,"""BBBBB""",538,"""fully, thick, light, limit, th…",false,"""linty""",0,null,"""very hard""","""soare""",1,"""linty""",1
"""yummy""","""linty""","""sitreaoln""","""y""",9,1,1,"""BBBBG""",50,"""fuzzy, puffy, guppy, dummy, mu…",false,"""fuzzy""",0,null,"""very hard""","""linty""",1,"""fuzzy""",1
"""yummy""","""fuzzy""","""sitreaofzln""","""yu""",11,2,2,"""BGBBG""",37,"""duchy, buddy, jumpy, mummy, yu…",true,"""duchy""",0,null,"""very hard""","""fuzzy""",1,"""duchy""",1
"""yummy""","""duchy""","""sitcreaofzdlhn""","""yu""",14,2,2,"""BGBBG""",16,"""puppy, mummy, buggy, bumpy, gu…",true,"""puppy""",0,null,"""very hard""","""duchy""",1,"""puppy""",1
"""yummy""","""puppy""","""sitcreaopfzdlhn""","""yu""",15,2,2,"""BGBBG""",9,"""mummy, buggy, gummy, muggy, yu…",true,"""mummy""",0,null,"""very hard""","""puppy""",1,"""mummy""",1


In [10]:
(
    simulating_tries_difficulty
    .unique(subset=["group"], keep="first")
    ["difficulty"]
    .value_counts()
    .sort("count", descending=True)
)

difficulty,count
str,u32
"""moderate""",5387
"""hard""",3069
"""very hard""",630
"""easy""",2


# Create dataframe of first guesses

In [11]:
# Group by "Category" and return the first row of each group
first_guess = (
    simulating_tries_difficulty
    .group_by("group", maintain_order=True)
    .first()
)

first_guess.head()

group,challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,tries,difficulty,guess_word_anagrams,anagram_num,anagrams,anagram_num_right
u32,str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u8,str,str,u8,str,u8
0,"""yummy""","""soare""","""sreao""","""""",5,0,0,"""BBBBB""",538,"""fully, thick, light, limit, th…",false,"""linty""",null,"""very hard""","""soare""",1,"""linty""",1
1,"""calyx""","""soare""","""sore""","""a""",4,1,0,"""BBYBB""",716,"""faint, fault, happy, naval, hu…",false,"""linty""",4,"""moderate""","""soare""",1,"""linty""",1
2,"""varas""","""soare""","""oe""","""sar""",2,3,0,"""YBYYB""",75,"""paris, marls, harsh, marsh, ar…",false,"""linty""",5,"""hard""","""soare""",1,"""linty""",1
3,"""bubba""","""soare""","""sore""","""a""",4,1,0,"""BBYBB""",716,"""cabin, naval, faint, daily, ch…",false,"""linty""",5,"""hard""","""soare""",1,"""linty""",1
4,"""lowes""","""soare""","""ar""","""soe""",2,3,1,"""YGBBY""",27,"""jones, moses, nosed, loess, eo…",true,"""linty""",4,"""moderate""","""soare""",1,"""linty""",1


In [12]:
avg_tries = first_guess["tries"].mean()
avg_tries

4.227240482383542

# EDA

In [13]:
# Plot difficulty distribution
difficulty_distribution(
    simulating_tries_difficulty
)

## Most frequent number of guesses

In [14]:
(
    first_guess
    ["tries"]
    .value_counts()
    .sort(by="count", descending=True)
)

tries,count
u8,u32
4,3329
5,2153
3,2058
6,916
null,630
1,1
2,1


### Challenge words without `r` , `s` or `t`
These are some of the most frequently ocuuring letters. We can then see how our naive algorithm operates in coming to the correct guess.

In [15]:
challenge_words_without_r_s_t = (
    first_guess
    .filter(
        ~(pl.col("challenge").str.contains("r|s|t"))
        & ((pl.col("challenge") != (pl.col("guess"))))
    )
)

challenge_words_without_r_s_t.sample(n = 20, with_replacement = False)

group,challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,tries,difficulty,guess_word_anagrams,anagram_num,anagrams,anagram_num_right
u32,str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u8,str,str,u8,str,u8
8389,"""bunje""","""soare""","""saor""","""e""",4,1,1,"""BBBBG""",284,"""piece, twice, white, venue, bi…",false,"""linty""",null,"""very hard""","""soare""",1,"""linty""",1
2825,"""boxen""","""soare""","""sar""","""oe""",3,2,1,"""BGBBY""",86,"""model, dozen, money, vowel, no…",false,"""linty""",null,"""very hard""","""soare""",1,"""linty""",1
993,"""bouge""","""soare""","""sar""","""oe""",3,2,2,"""BGBBG""",119,"""noble, movie, voice, lodge, lo…",false,"""linty""",5,"""hard""","""soare""",1,"""linty""",1
3644,"""dolce""","""soare""","""sar""","""oe""",3,2,2,"""BGBBG""",119,"""voice, donne, ponce, dodge, mo…",false,"""linty""",4,"""moderate""","""soare""",1,"""linty""",1
3352,"""block""","""soare""","""sare""","""o""",4,1,0,"""BYBBB""",237,"""blood, timon, flood, pilot, ou…",false,"""linty""",4,"""moderate""","""soare""",1,"""linty""",1
531,"""jemmy""","""soare""","""saor""","""e""",4,1,0,"""BBBBY""",444,"""level, field, belly, beech, de…",false,"""linty""",5,"""hard""","""soare""",1,"""linty""",1
6027,"""honan""","""soare""","""sre""","""oa""",3,2,1,"""BGYBB""",118,"""local, total, woman, vocal, to…",false,"""linty""",4,"""moderate""","""soare""",1,"""linty""",1
5595,"""yield""","""soare""","""saor""","""e""",4,1,0,"""BBBBY""",444,"""field, chief, event, tenth, ch…",true,"""linty""",3,"""moderate""","""soare""",1,"""linty""",1
4037,"""dwile""","""soare""","""saor""","""e""",4,1,1,"""BBBBG""",284,"""title, white, chile, chyle, tw…",false,"""linty""",null,"""very hard""","""soare""",1,"""linty""",1


In [16]:
# Plot difficulty distribution of of challenge words that doesn't contain 'r', 's' or 't'
difficulty_distribution(challenge_words_without_r_s_t)

## Null values (Incomplete games)

In [17]:
incomplete_games = (
    simulating_tries_difficulty
    .filter(pl.col("tries").is_null())
)

In [18]:
incomplete_games.head()

challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num,anagrams,anagram_num_right
str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u32,u8,str,str,u8,str,u8
"""yummy""","""soare""","""sreao""","""""",5,0,0,"""BBBBB""",538,"""fully, thick, light, limit, th…",false,"""linty""",0,null,"""very hard""","""soare""",1,"""linty""",1
"""yummy""","""linty""","""sitreaoln""","""y""",9,1,1,"""BBBBG""",50,"""fuzzy, puffy, guppy, dummy, mu…",false,"""fuzzy""",0,null,"""very hard""","""linty""",1,"""fuzzy""",1
"""yummy""","""fuzzy""","""sitreaofzln""","""yu""",11,2,2,"""BGBBG""",37,"""duchy, buddy, jumpy, mummy, yu…",true,"""duchy""",0,null,"""very hard""","""fuzzy""",1,"""duchy""",1
"""yummy""","""duchy""","""sitcreaofzdlhn""","""yu""",14,2,2,"""BGBBG""",16,"""puppy, mummy, buggy, bumpy, gu…",true,"""puppy""",0,null,"""very hard""","""duchy""",1,"""puppy""",1
"""yummy""","""puppy""","""sitcreaopfzdlhn""","""yu""",15,2,2,"""BGBBG""",9,"""mummy, buggy, gummy, muggy, yu…",true,"""mummy""",0,null,"""very hard""","""puppy""",1,"""mummy""",1


In [19]:
null_select_group = np.random.choice(incomplete_games["group"], 1, replace = False)[0]

HTML(
    incomplete_games
    .select(
        "challenge", "guess", "match_pattern", "letter_differences", "common_letters",
        "next_guess", "group", "tries", "difficulty", "guess_word_anagrams", "anagram_num"
    )
    .filter(pl.col("group") == null_select_group)
    .to_pandas()
    .assign(
        match_pattern = lambda df_:
        color_pattern_matching(
            dataf=df_,
            challenge_col="challenge",
            guess_col="guess"
        )
    )
    .to_html(escape=False)
)

,challenge,guess,match_pattern,letter_differences,common_letters,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num
0,feebs,soare,soare,aor,se,linty,5751,NaN,very hard,soare,1
1,feebs,linty,linty,itraolyn,se,jesus,5751,NaN,very hard,linty,1
2,feebs,jesus,jesus,itraojulyn,se,deevs,5751,NaN,very hard,jesus,1
3,feebs,deevs,deevs,itraovjudlyn,se,geeps,5751,NaN,very hard,deevs,1
4,feebs,geeps,geeps,itraovgpjudlyn,se,keeks,5751,NaN,very hard,geeps,1
5,feebs,keeks,keeks,itraovgpjkudlyn,se,feebs,5751,NaN,very hard,keeks,1


In [20]:
incomplete_games_check = (
    incomplete_games
    .filter(pl.col("group") == null_select_group)
    .select(
        "challenge", "guess", "match_pattern", "num_choices_after_guess",
        "possible_guesses", "challenge_in_possible_guesses", "next_guess"
    )
)

incomplete_games_check

challenge,guess,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess
str,str,str,u16,str,bool,str
"""feebs""","""soare""","""YBBBY""",156,"""lewis, exist, welsh, jesus, is…",false,"""linty"""
"""feebs""","""linty""","""BBBBB""",31,"""jesus, guess, chess, jukes, ps…",false,"""jesus"""
"""feebs""","""jesus""","""BGYBG""",13,"""deevs, kembs, hecks, kexes, ge…",false,"""deevs"""
"""feebs""","""deevs""","""BGGBG""",3,"""geeps, keeks, feebs""",true,"""geeps"""
"""feebs""","""geeps""","""BGGBG""",2,"""keeks, feebs""",true,"""keeks"""
"""feebs""","""keeks""","""BGGBG""",1,"""feebs""",true,"""feebs"""


In [21]:
(
    incomplete_games_check
    .gather_every(n=1)
    .head(1)
    ["possible_guesses"]
    .item()
)

'lewis, exist, welsh, jesus, islet, fetus, chest, penis, bless, upset'

In [22]:
(
    incomplete_games_check
    .gather_every(n=1, offset=1)
    .head(1)
    ["possible_guesses"]
    .item()
)

'jesus, guess, chess, jukes, pseud, chews, deevs, kembs, zebus, hecks'

In [23]:
(
    incomplete_games_check
    .gather_every(n=1, offset=2)
    .head(1)
    ["possible_guesses"]
    .item()
)

'deevs, kembs, hecks, kexes, geeps, fecks, keeks, meeds, kemps, femes'

In [24]:
(
    incomplete_games_check
    .gather_every(n=1, offset=3)
    .head(1)
    ["possible_guesses"]
    .item()
)

'geeps, keeks, feebs'

In [25]:
(
    incomplete_games_check
    .gather_every(n=1, offset=4)
    .head(1)
    ["possible_guesses"]
    .item()
)

'keeks, feebs'

In [26]:
(
    incomplete_games_check
    .gather_every(n=1, offset=5)
    .head(1)
    ["possible_guesses"]
    .item()
)

'feebs'

# Challenge word statistics

In [27]:
challenge_stats = (
    simulating_tries_difficulty
    .group_by("challenge")
    .agg(
        tries_mode=pl.col("tries").mode().cast(pl.UInt8),
        tries_mean=(pl.col("tries").mean().cast(pl.Float32)).round(3),
        tries_null_pct=(pl.col("tries").is_null().mean().cast(pl.Float32) * 100).round(3),
    )
    .explode("tries_mode")
    .sort("tries_null_pct", descending = True)
)

In [28]:
challenge_stats.head()

challenge,tries_mode,tries_mean,tries_null_pct
str,u8,f32,f32
"""emure""",null,null,100.0
"""hinny""",null,null,100.0
"""waver""",null,null,100.0
"""gawky""",null,null,100.0
"""laxer""",null,null,100.0


In [29]:
print(f"Overall incompletion rate: {(first_guess['tries'].is_null().sum() / first_guess.height) * 100:.2f}%")

Overall incompletion rate: 6.93%


## Retrieve the 2nd row of every group of 6 rows.

### Determine the average number of choices remaining after the second guess. This will tell us how effective the two guess strategy is at solving the challenge word.

In [30]:
second_guess_df = (
    simulating_tries_difficulty
    .with_row_index(name="row_num")
    .filter(
        (pl.col("row_num") % 6 == 1) &
        ~(pl.col("guess") == "saint") # remove instance where 'saint' as first guess is correct
    )
    .drop("row_num")
)

In [31]:
second_guess_df.head()

challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num,anagrams,anagram_num_right
str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u32,u8,str,str,u8,str,u8
"""yummy""","""linty""","""sitreaoln""","""y""",9,1,1,"""BBBBG""",50,"""fuzzy, puffy, guppy, dummy, mu…",false,"""fuzzy""",0,null,"""very hard""","""linty""",1,"""fuzzy""",1
"""calyx""","""linty""","""sitreon""","""aly""",7,3,0,"""YBBBY""",11,"""alkyl, calyx, phyla, allyl, by…",true,"""alkyl""",1,4,"""moderate""","""linty""",1,"""alkyl""",1
"""varas""","""linty""","""iteolyn""","""sar""",7,3,0,"""BBBBB""",32,"""marsh, harsh, argus, ramus, va…",false,"""marsh""",2,5,"""hard""","""linty""",1,"""marsh""",1
"""bubba""","""linty""","""sitreolyn""","""a""",9,1,0,"""BBBBB""",59,"""madam, bubba, gamma, pampa, ka…",true,"""madam""",3,5,"""hard""","""linty""",1,"""madam""",1
"""lowes""","""linty""","""tirayn""","""soel""",6,4,2,"""GBBBB""",4,"""loess, lowes, losel, lomes""",true,"""loess""",4,4,"""moderate""","""linty""",1,"""loess""",1


In [32]:
# More efficiently 
second_guess_df_2 = (
    simulating_tries_difficulty
    .gather_every(n=6, offset=1)
    .filter(~(pl.col("guess") == "saint")) # remove instance where 'saint' as first guess is correct
)

second_guess_df_2.head()

challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num,anagrams,anagram_num_right
str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u32,u8,str,str,u8,str,u8
"""yummy""","""linty""","""sitreaoln""","""y""",9,1,1,"""BBBBG""",50,"""fuzzy, puffy, guppy, dummy, mu…",false,"""fuzzy""",0,null,"""very hard""","""linty""",1,"""fuzzy""",1
"""calyx""","""linty""","""sitreon""","""aly""",7,3,0,"""YBBBY""",11,"""alkyl, calyx, phyla, allyl, by…",true,"""alkyl""",1,4,"""moderate""","""linty""",1,"""alkyl""",1
"""varas""","""linty""","""iteolyn""","""sar""",7,3,0,"""BBBBB""",32,"""marsh, harsh, argus, ramus, va…",false,"""marsh""",2,5,"""hard""","""linty""",1,"""marsh""",1
"""bubba""","""linty""","""sitreolyn""","""a""",9,1,0,"""BBBBB""",59,"""madam, bubba, gamma, pampa, ka…",true,"""madam""",3,5,"""hard""","""linty""",1,"""madam""",1
"""lowes""","""linty""","""tirayn""","""soel""",6,4,2,"""GBBBB""",4,"""loess, lowes, losel, lomes""",true,"""loess""",4,4,"""moderate""","""linty""",1,"""loess""",1


In [33]:
print(f'Avg num choices after 2nd guess: {round(second_guess_df_2["num_choices_after_guess"].mean(), 2)}%')

Avg num choices after 2nd guess: 16.22%


### Greatest num of choices left after second guess

In [34]:
max_num_choices_after_2nd_guess = (
    second_guess_df_2
    .sort("num_choices_after_guess", descending=True)
    ["num_choices_after_guess"]
    .head(1)
    .item()
)

max_num_choices_after_2nd_guess

101

In [35]:
(
    second_guess_df_2
    .filter(pl.col("num_choices_after_guess") == max_num_choices_after_2nd_guess)
    ["challenge"]
    .unique()
    .to_numpy()
)


array(['areca', 'agger', 'rarer', 'wager', 'remap', 'baker', 'aread',
       'debar', 'afear', 'razer', 'feuar', 'raker', 'hazer', 'gamer',
       'maker', 'whear', 'freak', 'wreak', 'begar', 'farer', 'carer',
       'caber', 'bream', 'ardeb', 'arepa', 'herma', 'rehab', 'varec',
       'wader', 'recap', 'derma', 'jager', 'dazer', 'rebar', 'macer',
       'hepar', 'racer', 'raver', 'eggar', 'dream', 'caper', 'parer',
       'raper', 'paper', 'perea', 'faver', 'gager', 'rawer', 'barer',
       'gaper', 'wafer', 'zerda', 'regma', 'pawer', 'rewax', 'haver',
       'amber', 'aredd', 'gazer', 'creak', 'faker', 'carex', 'cedar',
       'auger', 'kerma', 'armed', 'waxer', 'daker', 'eager', 'japer',
       'cager', 'paver', 'pager', 'warez', 'ameer', 'fader', 'waver',
       'caver', 'mazer', 'pacer', 'adder', 'waker', 'arear', 'parev',
       'cream', 'gerah', 'pareu', 'acker', 'bread', 'dewar', 'kebar',
       'break', 'facer', 'drear', 'regar', 'abear', 'embar', 'armer',
       'harem', 'ra

### How often on average is the challenge word in the possible choices after the second guess

In [36]:
print(f'Avg times challenge in possible guesses: {round(second_guess_df_2["challenge_in_possible_guesses"].mean() * 100, 2)}%')

Avg times challenge in possible guesses: 74.74%


In [43]:
(
    simulating_tries_difficulty
    .filter(pl.col("challenge") == "hound")
)

challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num,anagrams,anagram_num_right
str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u32,u8,str,str,u8,str,u8
"""hound""","""soare""","""sare""","""o""",4,1,1,"""BGBBB""",340,"""young, mouth, point, youth, do…",false,"""linty""",2248,6,"""hard""","""soare""",1,"""linty""",1
"""hound""","""linty""","""sitrealy""","""on""",8,2,1,"""BBYBB""",12,"""wound, pound, mound, hound, co…",true,"""wound""",2248,6,"""hard""","""linty""",1,"""wound""",1
"""hound""","""wound""","""sitreawly""","""onud""",9,4,4,"""BGGGG""",3,"""mound, pound, hound""",true,"""mound""",2248,6,"""hard""","""wound""",1,"""mound""",1
"""hound""","""mound""","""sitmreawly""","""onud""",10,4,4,"""BGGGG""",2,"""pound, hound""",true,"""pound""",2248,6,"""hard""","""mound""",1,"""pound""",1
"""hound""","""pound""","""sitmreapwly""","""onud""",11,4,4,"""BGGGG""",1,"""hound""",true,"""hound""",2248,6,"""hard""","""pound""",1,"""hound""",1
"""hound""","""hound""","""""","""onudh""",0,5,5,"""GGGGG""",0,"""""",false,"""hound""",2248,6,"""hard""","""hound""",1,"""hound""",1


In [44]:
(
    simulating_tries_difficulty
    .filter(pl.col("challenge") == "fizzy")
)

challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num,anagrams,anagram_num_right
str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u32,u8,str,str,u8,str,u8
"""fizzy""","""soare""","""sreao""","""""",5,0,0,"""BBBBB""",538,"""dutch, night, civic, ninth, th…",false,"""linty""",8499,5,"""hard""","""soare""",1,"""linty""",1
"""fizzy""","""linty""","""streaoln""","""iy""",8,2,2,"""BGBBG""",41,"""jimmy, giddy, micky, dizzy, pi…",false,"""jimmy""",8499,5,"""hard""","""linty""",1,"""jimmy""",1
"""fizzy""","""jimmy""","""stmreaojln""","""iy""",10,2,2,"""BGBBG""",29,"""dizzy, vichy, giddy, biddy, pi…",false,"""dizzy""",8499,5,"""hard""","""jimmy""",1,"""dizzy""",1
"""fizzy""","""dizzy""","""stmreaojdln""","""iyz""",11,3,3,"""BGGGG""",2,"""fizzy, bizzy""",true,"""fizzy""",8499,5,"""hard""","""dizzy""",1,"""fizzy""",1
"""fizzy""","""fizzy""","""""","""iyzf""",0,4,4,"""GGGGG""",0,"""""",false,"""fizzy""",8499,5,"""hard""","""fizzy""",1,"""fizzy""",1
"""fizzy""","""fizzy""","""""","""iyzf""",0,4,4,"""GGGGG""",0,"""""",false,"""fizzy""",8499,5,"""hard""","""fizzy""",1,"""fizzy""",1
